In [ ]:
PATH = "/home/xjaros2/Documents/git/csidh-setup/"
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F3'

SS_VER = 'SS_VER_2_1'
attack_type = "A1"

In [ ]:
%cd $PATH/notebooks
%run ./init.ipynb
%cd $PATH/notebooks
# csidh.voltage_glitching_setup()
csidh.scope.cglitch_setup()
csidh.action_sleep = 0.25
%cd $PATH/notebooks

In [ ]:
csidh.reset_target()
csidh.scope.arm()
csidh.action()
ret = csidh.scope.capture()
if ret:
    print("Timeout happened during acquisition")
PUBLIC_EXPECTED = csidh.public_with_errors
MAX_EXT_OFFSET = csidh.scope.adc.trig_count

print("#" * 80)
print(f"PUBLIC: {PUBLIC_EXPECTED}")
print(f"TRIGS:  {MAX_EXT_OFFSET}")
print("#" * 80)

In [ ]:
# These need to be measured manually, will be used in the figures
# Also, bit of cheating here, since this means we know what degree isogeny is where
dummy_max_ext_offset = 4705896
dummy_isogeny_data = [
    {"degree": 3, "setup_start": 0,      "isogeny_start": 267412,  "isogeny_end": 327688},
    {"degree": 5, "setup_start": 327752, "isogeny_start": 447160,  "isogeny_end": 531468},
    {"degree": 7, "setup_start": 531500, "isogeny_start": 619624,  "isogeny_end": 675316},
    {"degree": 3, "setup_start": 675332, "isogeny_start": 1114596, "isogeny_end": 1175536},
    {"degree": 5, "setup_start": 1175588,"isogeny_start": 1295864, "isogeny_end": 1380000},
    {"degree": 7, "setup_start": 1380024,"isogeny_start": 1466320, "isogeny_end": 1523632},
    {"degree": 5, "setup_start": 1523672,"isogeny_start": 2083708, "isogeny_end": 2168156},
    {"degree": 3, "setup_start": 2168232,"isogeny_start": 2694412, "isogeny_end": 2755560},
    {"degree": 7, "setup_start": 2755604,"isogeny_start": 2962932, "isogeny_end": 3020256},
    {"degree": 3, "setup_start": 3020396,"isogeny_start": 3457520, "isogeny_end": 3518488},
    {"degree": 5, "setup_start": 3518528,"isogeny_start": 3638024, "isogeny_end": 3729888},
    {"degree": 7, "setup_start": 3730068,"isogeny_start": 3815672, "isogeny_end": 3873328},
    {"degree": 3, "setup_start": 3873524,"isogeny_start": 4214548, "isogeny_end": 4275180},
    {"degree": 5, "setup_start": 4275208,"isogeny_start": 4395016, "isogeny_end": 4486364},
    {"degree": 7, "setup_start": 4486600,"isogeny_start": 4573456, "isogeny_end": 4631104},
]

In [ ]:
assert dummy_max_ext_offset == MAX_EXT_OFFSET
assert len(dummy_isogeny_data) == 15

In [ ]:
from csidhtools import Unit
def population_from_base_unit(base_unit, ext_offsets):
    units = []
    for ext_offset in ext_offsets:
        unit = Unit()
        unit.offset = base_unit.offset
        unit.width = base_unit.width
        unit.repeat = base_unit.repeat
        unit.ext_offset = ext_offset
        units.append(unit)
    return units

def split_into_batches(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [ ]:
import numpy as np
from csidhtools.search.ga import evaluate_unit_default, cache
# Number of measurements per isogeny
MEASUREMENTS_PER_ISOGENY = 50000
# Set the measurements per unit
MEASUREMENTS_PER_UNIT = 1
evaluate_unit = lambda csidh, unit: evaluate_unit_default(csidh, unit, MEASUREMENTS_PER_UNIT)
# Set the base unit
BASE_UNIT = Unit()
BASE_UNIT.offset = 216
BASE_UNIT.width  = 285
BASE_UNIT.repeat = 14
# Create cache
CACHE_INTERVAL = 10
# Output filename
FILENAME = "husky-clock-isogeny-single-glitches-three-150k.json"

population = []
for i in [4, 6, 7]:
    l = dummy_isogeny_data[i]["setup_start"]
    r = dummy_isogeny_data[i]["isogeny_end"]
    population += population_from_base_unit(BASE_UNIT, np.linspace(l, r, MEASUREMENTS_PER_ISOGENY).astype(int))
len(population)

for batch in tqdm(list(split_into_batches(population, CACHE_INTERVAL))):
    evaluate_batch(csidh, batch, evaluate_unit)
    write_cache_to_file(FILENAME, cache)

In [ ]:
csidh.dis()

In [ ]:
# potential = [
#     "./husky-clock-xISOG-first-5-isogeny-skip-search-until7.json"
# ]

# df = read_cachefiles_to_dataframe(potential)
# df = df[(df["type"] == "JUSTRIGHT")].reset_index()
# indices = [i  for i,x in enumerate(df["responses"]) if 199 in x]
# potential_skips = df.iloc[indices]
# batch = list(set(potential_skips["unit"]))
# batch.sort(key=lambda unit: -unit.ext_offset)
# batch
# cache = OrderedDict()
# uut = batch[-1]
# uut.offset = 2706
# uut.width = 2600
# uut.repeat = 4
# evaluate_batch(csidh, [uut])
# write_cache_to_file(f"first-5-isogeny-potential-skips.json", cache, 1, len(cache), -1)   

In [ ]:
# cache = OrderedDict()
# REPEATS = 20000
# cache_interval = 2
# for _ in tqdm(range(REPEATS//cache_interval)):
#     batch = []
#     for _ in range(cache_interval):
#         i = random.randint(0, len(candidates)-1)
#         base_unit = candidates[i]
#         for j in range(len(isogeny_offsets)):
#             if j == 0:
#                 eof_min = 0
#                 eof_max = isogeny_offsets[0][1]
#             else:
#                 eof_min = isogeny_offsets[j-1][2]
#                 eof_max = isogeny_offsets[j][1]
#             unit = Unit(repr=repr(base_unit))
#             unit.ext_offset = random.randint(eof_min, eof_max)
#             print(unit)
#             batch.append(unit)

#     evaluate_batch(csidh, batch)
#     write_cache_to_file(f"husky-clock-ISOGENY-SKIP-SEARCH.json", cache, 1, len(cache), -1)   

In [ ]:
# cache = OrderedDict()

# REPEATS = 17500
# cache_interval = 10
# for _ in tqdm(range(REPEATS//cache_interval)):
#     batch = []
#     for _ in range(cache_interval):
#         # width = random.randint(2650, 2750)
#         # offset = random.randint(2560, 2610)
#         # repeat = random.randint(4, 5)
#         width = random.randint(0, csidh.scope.glitch.phase_shift_steps//2)
#         offset = random.randint(0, csidh.scope.glitch.phase_shift_steps)
#         repeat = random.randint(1, 18)
        
#         ext_offset = random.randint((474252//4 )+2000, (474252//4 ) + 3200)
    
#         unit = Unit(repr=f"{ext_offset},{offset},{width},{repeat},None,0")
#         batch.append(unit)

#     evaluate_batch(csidh, batch)
#     write_cache_to_file(f"husky-clock-xISOG-first-5-isogeny-skip-parameter-search.json", cache, 1, len(cache), -1)   